In [1]:
with open("Day09.txt") as file:
    data = list(map(int, file.read()))

In [2]:
from collections import defaultdict

def get_memory(data):
    file, total = 0, 0
    file_blocks, empty_blocks = defaultdict(list), []
    for index, size in enumerate(data):
        if not size:
            continue
        blocks = list(range(total, total + size))
        if index % 2:
            empty_blocks.append(blocks)
        else:
            file_blocks[file] = blocks
            file += 1
        total += size
    return file_blocks, empty_blocks

def get_checksum(file_blocks):
    return sum(file * block for file, blocks in file_blocks.items() for block in blocks)

In [3]:
from bisect import insort
from queue import deque

def defrag(data):
    file_blocks, empty_blocks = get_memory(data)
    empties = deque(block for blocks in empty_blocks for block in blocks)
    for file, blocks in sorted(file_blocks.items(), reverse=True):
        for index, block in enumerate(blocks):
            if block < empties[0]:
                return get_checksum(file_blocks)
            blocks[index] = empties.popleft()
            insort(empties, block)

In [4]:
%%time
defrag(data)

CPU times: user 172 ms, sys: 3.97 ms, total: 176 ms
Wall time: 175 ms


6356833654075

In [5]:
def defrag_blocks(data):
    file_blocks, empty_blocks = get_memory(data)
    for file, blocks in sorted(file_blocks.items(), reverse=True):
        file_size = len(blocks)
        for index, empty_block in enumerate(empty_blocks):
            empty_size = len(empty_block)
            if file_size > empty_size:
                continue
            if not empty_block or blocks[0] < empty_block[0]:
                break
            blocks[:] = empty_block[:file_size]
            empty_block[:] = empty_block[file_size:]
            if not empty_block:
                del empty_blocks[index]
            break
    return get_checksum(file_blocks)

In [6]:
%%time
defrag_blocks(data)

CPU times: user 78.3 ms, sys: 1.96 ms, total: 80.3 ms
Wall time: 80.1 ms


6389911791746